# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

weather_data_file = "cities.csv"
weather_df = pd.read_csv(weather_data_file)
weather_df.head()

,Unnamed: 0,City ID,City Name,Cloudiness,Country,Date,Humidity,Latitiude,Longitude,Max Temp,Wind Speed
0,0,[0],ribeira grande,91,PT,1665537239,89,38.5167,-28.7000,69.78,1.99
1,1,[1],honningsvag,100,NO,1665537239,71,70.9821,25.9704,46.15,13.80
2,2,[2],punta arenas,75,CL,1665537155,81,-53.1500,-70.9167,39.31,18.41
3,3,[3],salalah,0,OM,1665537240,49,17.0151,54.0924,73.49,2.30
4,4,[4],qaanaaq,44,GL,1665537240,62,77.4840,-69.3632,5.79,7.85


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Plot Heatmap
gmaps.configure(api_key = g_key)

fig = gmaps.figure()
locations = weather_df[["Latitiude", "Longitude"]]
 
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weather_df["Humidity"], 
                                 dissipating=False,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.
hotel_df = weather_df[(weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70) & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0)] 


hotel_df.head()
len(hotel_df)

16

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [65]:
pd.options.mode.chained_assignment = None  # default='warn'
hotel_df['Hotel Name'] = ""
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    lat : index["Latitiude"]
    lon : index["Longitude"]
    params = {
        "location" : "{lat},{lon}"
        "rankby": "distance",
        "type": "lodging",
        "key": g_key
    }
    response = requests.get(base_url, params=params ).json()
    results = response['results']
    try:
        hotel_df.loc[index, "Hotel Name"] = results[8]['name']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


IndexError: list index out of range

In [62]:
hotel_df.head()

,Unnamed: 0,City ID,City Name,Cloudiness,Country,Date,Humidity,Latitiude,Longitude,Max Temp,Wind Speed,Hotel Name
3,3,[3],salalah,0,OM,1665537240,49,17.0151,54.0924,73.49,2.30,
33,33,[34],gat,0,IL,1665537249,100,31.6100,34.7642,71.40,3.31,
72,72,[78],awbari,0,LY,1665537263,27,26.5921,12.7805,70.72,6.53,
135,135,[143],batesville,0,US,1665537275,49,35.7698,-91.6410,74.08,4.61,
164,164,[176],luxor,0,EG,1665537291,41,25.6989,32.6421,72.77,5.70,


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'City'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
